# Get Citibike trip data

In [1]:
import os
import requests
import pandas as pd
import numpy as np
from tqdm import tqdm
%matplotlib inline

In [2]:
%%time
#output.to_csv("hourly_trajectories.csv")

CPU times: user 1 µs, sys: 1e+03 ns, total: 2 µs
Wall time: 4.05 µs


# Frame the data as a machine learning problem

In [3]:
data = pd.read_csv("hourly_trajectories.csv")

In [4]:
data.index = data['Unnamed: 0']

In [5]:
del data['Unnamed: 0']

In [6]:
del data.index.name

In [7]:
data.head()

,volume,trajectory
2018-01-01 00:00:00,0.0,"(119, 119)"
2018-01-01 01:00:00,0.0,"(119, 119)"
2018-01-01 02:00:00,0.0,"(119, 119)"
2018-01-01 03:00:00,0.0,"(119, 119)"
2018-01-01 04:00:00,0.0,"(119, 119)"


In [8]:
%%time
data['volume_t-1'] = data['volume'].shift(1)
data['volume_t-2'] = data['volume'].shift(2)
data['volume_t-3'] = data['volume'].shift(3)
data['volume_t-4'] = data['volume'].shift(4)
data['volume_t-5'] = data['volume'].shift(5)
data['volume_t-6'] = data['volume'].shift(6)
data['volume_t-7'] = data['volume'].shift(7)
data['volume_t-8'] = data['volume'].shift(8)
data['volume_t-9'] = data['volume'].shift(9)
data['volume_t-10'] = data['volume'].shift(10)
data['volume_t-11'] = data['volume'].shift(11)
data['volume_t-12'] = data['volume'].shift(12)
data['volume_t-13'] = data['volume'].shift(13)
data['volume_t-14'] = data['volume'].shift(14)
data['volume_t-15'] = data['volume'].shift(15)
data['volume_t-16'] = data['volume'].shift(16)
data['volume_t-17'] = data['volume'].shift(17)
data['volume_t-18'] = data['volume'].shift(18)
data['volume_t-19'] = data['volume'].shift(19)
data['volume_t-20'] = data['volume'].shift(20)
data['volume_t-21'] = data['volume'].shift(21)
data['volume_t-22'] = data['volume'].shift(22)
data['volume_t-23'] = data['volume'].shift(23)
data['volume_t-24'] = data['volume'].shift(24)

CPU times: user 10.2 s, sys: 14.8 s, total: 24.9 s
Wall time: 27.2 s


In [ ]:
X = data[['volume_t-1','volume_t-2','volume_t-3','volume_t-4','volume_t-5','volume_t-6','volume_t-7', \
    'volume_t-8','volume_t-9','volume_t-10','volume_t-11','volume_t-12','volume_t-13','volume_t-14', \
    'volume_t-15','volume_t-16','volume_t-17','volume_t-18','volume_t-19','volume_t-20','volume_t-21', \
    'volume_t-22','volume_t-23','volume_t-24']]

In [ ]:
X.head()

In [ ]:
X.tail()

In [ ]:
y = data[['volume']]

In [ ]:
y.head()

In [ ]:
y.tail()

# Scale the data

In [ ]:
scaler_X = MinMaxScaler(feature_range=(0, 1))
X = np.array(X)
X = scaler_X.fit_transform(X)

In [ ]:
scaler_y = MinMaxScaler(feature_range=(0, 1))
y = np.array(y).reshape((len(y), 1))
y = scaler_y.fit_transform(y)

In [ ]:
X.shape

In [ ]:
y.shape

# Split the data into training and testing

In [ ]:
n_train_hours = 24 * 25

In [ ]:
X_train = X[0 : n_train_hours, ]
X_test = X[n_train_hours:, ]

# Reshape
X_train = X_train.reshape(X_train.shape + (1,))
X_test = X_test.reshape(X_test.shape + (1,))

In [ ]:
y_train = y[0 : n_train_hours]
y_test = y[n_train_hours:]

In [ ]:
print X_train.shape[0], "examples"
print X_train.shape[1], "features"
print X_train.shape[2], "time lags"

# The model

In [ ]:
seed = 7

In [ ]:
np.random.seed(seed)

In [ ]:
model = Sequential()
model.add(LSTM(50, input_shape=(X_train.shape[1], X_train.shape[2])))
model.add(Dense(1))
model.compile(loss='mae', optimizer='adam')

# fit network
history = model.fit(X_train, y_train, epochs=100, batch_size=50,
                    validation_data=(X_test, y_test), verbose=2, shuffle=False)

# Get list of Citibike stations

In [ ]:
stations_url = "https://feeds.citibikenyc.com/stations/stations.json"
stations_response = requests.get(stations_url)
stations_json = stations_response.json()

In [ ]:
station_ids = []
station_lats = []
station_lons = []
station_names = []
station_test = []
station_docks = []

for i in stations_json['stationBeanList']:
    station_ids.append(i['id'])
    station_lats.append(i['latitude'])
    station_lons.append(i['longitude'])
    station_names.append(i['stationName'])
    station_test.append(i['testStation'])
    station_docks.append(i['totalDocks'])
    
stations = pd.DataFrame()
stations['id'] = station_ids
stations['latitude'] = station_lats
stations['longitude'] = station_lons
stations['name'] = station_names
stations['test'] = station_test
stations['docks'] = station_docks

stations.sort_values(by='id', inplace=True)
stations.index=stations['id']
del stations['id']

In [ ]:
stations.head()